# Practical Decorators 

From PyCon 2019 https://youtu.be/MjHpMCIvwsY 

Think this way. 

```python
@mydeco
def add(a+b):
    return a+b
```

equals to

```python
def add(a+b):
    return a+b

add = mydeco(add)
```

In [2]:
def mydeco(func): # Only runs when we decorate the function. 
    def wrapper(*args, **kwargs): # Runs every time the decorated function is called. 
        return f'{func(*args, **kwargs)} !!!'
    return wrapper

In [5]:
import time

def logtime(func):
    def wrapper(*args, **kwargs):
        start_time = time.time()
        result = func(*args, **kwargs)
        total_time = time.time() - start_time
        
        print(f'{time.time()} \t {func.__name__} \t {total_time}')
        
        return result
    
    return wrapper
    

In [6]:
@logtime
def add(a, b):
    return a + b

In [7]:
add(10, 10)

1608983846.4999652 	 add 	 0.0


20


Adding argument to a decorator은 여기서부터 나온다. 

https://youtu.be/MjHpMCIvwsY?t=721

In [23]:
class CalledTooOftenError(Exception):
    def __init__(self, msg):
        super().__init__(msg)

In [24]:
def once_per_n(n):
    def middle(func):
        last_invoked = 0
            
        def wrapper(*args, **kwargs):
            nonlocal last_invoked
            elapsed_time = time.time() - last_invoked
            if elapsed_time < n:
                raise CalledTooOftenError(f"Only {elapsed_time} has passed.")

            last_invoked = time.time()
            return func(*args, **kwargs)

        return wrapper
    
    return middle

In [25]:
import time

@once_per_n(n=60)
def slow_add(a, b):
    time.sleep(10)
    return a+b


In [26]:
slow_add(10, 20)

30

In [27]:
slow_add(20, 30)

CalledTooOftenError: Only 10.020977258682251 has passed.

Memoization 용도로도 쓸 수 있다. 


In [9]:
def memoize(func):
    cache = {}
    
    def wrapper(*args, **kwargs):
        if args not in cache:
            print(f'Caching NEW value for {func.__name__}{args}')
            cache[args] = func(*args, **kwargs)
        else:
            print(f'Using OLD(cached) value for {func.__name__}{args}')
        return cache[args]
    return wrapper

`args`가 non-hashable한 경우나 `kwargs`를 처리해야 하는 경우는? 

strings and bytestrings는 hashable하니 `pickle`시킨다. 

So use a tuple of bytestrings as your dict keys and you'll be find for most purposes

In [1]:
import pickle

def memoize(func):
    cache = {}
    
    def wrapper(*args, **kwargs):
        t = (pickle.dumps(args), pickle.dumps(kwargs))
        if t not in cache:
            print(f"Caching NEW value for {func.__name__}{args}")
            cache[t] = func(*args, **kwargs)
        else:
            print(f"Using OLD value for {func.__name__}{args}")
        
        return cache[t]
    
    return wrapper

Attributes 에도 사용 가능. 

서로 다른 class들에게 모두 attributes를 부여해야 할 때도 유용하게 사용 가능.

상속도 attributes를 공유할 수 있지만 상속은 상속 관계 있는 class들에게 쓰는 것이기에 적절치 않음. (no multiple inheritance)

In [28]:
def object_birthday(c):
    def wrapper(*args, **kwargs):
        o = c(*args, **kwargs)
        o._created_at = time.time()
        return o
    return wrapper